# Week 1: Find Clusters of Infected People

<span style="color:red">
**URGENT WARNING**

We have been receiving reports from health facilities that a new, fast-spreading virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Your goal for this notebook will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [1]:
import cudf
import cuml

import cupy as cp

## Load Data

Begin by loading the data you've received about week 1 of the outbreak into a cuDF data frame. The data is located at `'./data/week1.csv'`. For this notebook you will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after loading, or use the `cudf.read_csv` named argument `usecols` to provide a list of only the columns you need.

In [2]:
df = cudf.read_csv('./data/week1.csv', usecols = ['lat', 'long', 'infected'])


In [3]:
df.head()

,lat,long,infected
0,54.522510,-1.571896,False
1,54.554030,-1.524968,False
2,54.552486,-1.435203,False
3,54.537189,-1.566215,False
4,54.528212,-1.588462,False


## Make Data Frame of the Infected

Make a new cuDF data frame `infected_df` that contains only the infected members of the population.

In [4]:
infected_df = df[df['infected'] == True]
infected_df.head()

,lat,long,infected
28928759,54.472766,-1.654932,True
28930512,54.529717,-1.667143,True
28930904,54.512986,-1.589866,True
28932226,54.522322,-1.380694,True
28933748,54.541660,-1.613490,True


## Make Grid Coordinates for Infected Locations

Provided for you in the next cell (which you can expand by clicking on the "..." and contract again after executing by clicking on the blue left border of the cell) is the lat/long to OSGB36 grid coordinates converter you used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `infected_df` you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [17]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
infected_df['northing'] = cudf.Series(n_cupy_array).astype('float32')
infected_df['easting'] = cudf.Series(e_cupy_array).astype('float32')
print(infected_df.dtypes)
infected_df.head()

index         int64
lat         float64
long        float64
infected       bool
northing    float32
easting     float32
dtype: object


,index,lat,long,infected,northing,easting
0,28928759,54.472766,-1.654932,True,508670.06250,422359.7500
1,28930512,54.529717,-1.667143,True,515002.65625,421538.5625
2,28930904,54.512986,-1.589866,True,513167.53125,426549.8750
3,28932226,54.522322,-1.380694,True,514305.28125,440081.2500
4,28933748,54.541660,-1.613490,True,516349.12500,425003.0000


In [18]:
infected_df.head()

,index,lat,long,infected,northing,easting
0,28928759,54.472766,-1.654932,True,508670.06250,422359.7500
1,28930512,54.529717,-1.667143,True,515002.65625,421538.5625
2,28930904,54.512986,-1.589866,True,513167.53125,426549.8750
3,28932226,54.522322,-1.380694,True,514305.28125,440081.2500
4,28933748,54.541660,-1.613490,True,516349.12500,425003.0000


## Find Clusters of Infected People

Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member. Create a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [19]:
dbscan = cuml.DBSCAN(eps = 2000, min_samples = 25)
#infected_df = df[df['infected'] == True].reset_index()
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()
infected_df['cluster'].count()

18148

## Find the Centroid of Each Cluster

Use grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [20]:
clusters = infected_df[['cluster', 'northing', 'easting']].groupby(['cluster'])
clusters.mean()

,northing,easting
cluster,,
10,334208.218750,435937.750000
6,434970.407407,406985.296296
11,300567.970588,391901.529412
9,417322.476190,409583.761905
4,391630.060606,431158.181818
-1,378085.912179,401876.792993
5,386471.232558,426559.069767
1,436475.500000,332980.411765
13,289854.845070,394518.225352


Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [21]:
infected_df['cluster'].value_counts()

 0     8638
-1     8449
 2      403
 8       94
 12      72
 13      71
 11      68
 1       68
 4       66
 10      64
 5       43
 7       39
 6       27
 3       25
 9       21
Name: cluster, dtype: int32

In [58]:
dbscan = cuml.DBSCAN(eps = 2000, min_samples = 25)
infected_df2 = df[df['infected'] == True].reset_index()
infected_df2['cluster'] = dbscan.fit_predict(infected_df2[['northing', 'easting']])
infected_df2['cluster'].nunique()
infected_df2['cluster'].count()

clusters = infected_df2[['cluster', 'northing', 'easting']].groupby(['cluster'])
clusters.mean()

#

,northing,easting
cluster,,
10,300433.079365,391700.507937
6,434970.333333,406985.259259
11,291518.957746,401687.774648
9,334235.873016,435895.555556
4,391308.416667,431463.375000
-1,378229.513074,401626.842069
5,387248.774194,426614.419355
1,427035.206897,366582.206897
8,416023.500000,414675.772727


In [59]:
infected_df2['cluster'].value_counts()

-1     8605
 0     8558
 3      396
 8       88
 11      71
 12      66
 2       65
 10      63
 9       63
 4       48
 7       38
 5       31
 1       29
 6       27
Name: cluster, dtype: int32

## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 1 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page.

There will be additional questions for you to answer after completing the remaining notebooks. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Please Restart the Kernel</h2></div>

In [22]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}